# US Treasury Data

In [1]:
import pandas as pd
import pandas_datareader as pdr
import pandas_datareader as pdr
import plotly.graph_objects as go
import plotly.express as px
from scipy import stats
from helpers.calculators import calculate_pct_changes, rolling_columns_change_array, calculate_rolling_z_scores, months_array

In [2]:
# Get US Treasury Rates from FRED

start = '2000-01-01'
tickers = ['GS30','GS10','GS5','GS3','GS2','GS1','GS6m','GS3m']
treasury_df = pdr.get_data_fred(tickers,start)
treasury_df.columns=['30Y','10Y','5Y','3Y','2Y','1Y','6M','3M']
treasury_df.dropna(inplace=True)
# Changing format from 1st day of the month to last day of the month
treasury_df.index = treasury_df.index + pd.offsets.MonthEnd(0)
print(f'rates as at: {treasury_df.index[-1]}')

rates as at: 2023-06-30 00:00:00


In [3]:
treasury_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 282 entries, 2000-01-31 to 2023-06-30
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   30Y     282 non-null    float64
 1   10Y     282 non-null    float64
 2   5Y      282 non-null    float64
 3   3Y      282 non-null    float64
 4   2Y      282 non-null    float64
 5   1Y      282 non-null    float64
 6   6M      282 non-null    float64
 7   3M      282 non-null    float64
dtypes: float64(8)
memory usage: 19.8 KB


In [4]:
   fig = go.FigureWidget(data=[go.Surface(x=treasury_df.columns,
                                    y=treasury_df.index,
                                    z=treasury_df.values,
                                    opacity=0.95,
                                    connectgaps=True,
                                    colorscale='rdbu',
                                    showscale=True,
                                    reversescale=True,
                                    )
                        ]
                )
fig


FigureWidget({
    'data': [{'colorscale': [[0.0, 'rgb(103,0,31)'], [0.1, 'rgb(178,24,43)'],
                             [0.2, 'rgb(214,96,77)'], [0.3, 'rgb(244,165,130)'],
                             [0.4, 'rgb(253,219,199)'], [0.5, 'rgb(247,247,247)'],
                             [0.6, 'rgb(209,229,240)'], [0.7, 'rgb(146,197,222)'],
                             [0.8, 'rgb(67,147,195)'], [0.9, 'rgb(33,102,172)'],
                             [1.0, 'rgb(5,48,97)']],
              'connectgaps': True,
              'opacity': 0.95,
              'reversescale': True,
              'showscale': True,
              'type': 'surface',
              'uid': 'd40635f1-640e-454a-8b60-c71f62b79310',
              'x': array(['30Y', '10Y', '5Y', '3Y', '2Y', '1Y', '6M', '3M'], dtype=object),
              'y': array([datetime.datetime(2000, 1, 31, 0, 0),
                          datetime.datetime(2000, 2, 29, 0, 0),
                          datetime.datetime(2000, 3, 31, 0, 0), ...,
        

In [5]:
treasury_df["10_v_2"] = (treasury_df["10Y"]- treasury_df["2Y"])

In [6]:
fig = go.FigureWidget(data=[go.Scatter(x=treasury_df.index,
                                y=treasury_df["10_v_2"] * 100,
                                mode ='lines',
                                )
                    ]
            )

fig.update_layout(
yaxis=dict(
    title="10yr - 2yr Tsry Yield",
    ticksuffix=" bps"
)
)


fig


FigureWidget({
    'data': [{'mode': 'lines',
              'type': 'scatter',
              'uid': 'a01d5c04-bdc1-4695-acb5-32d72daff3f6',
              'x': array([datetime.datetime(2000, 1, 31, 0, 0),
                          datetime.datetime(2000, 2, 29, 0, 0),
                          datetime.datetime(2000, 3, 31, 0, 0), ...,
                          datetime.datetime(2023, 4, 30, 0, 0),
                          datetime.datetime(2023, 5, 31, 0, 0),
                          datetime.datetime(2023, 6, 30, 0, 0)], dtype=object),
              'y': array([ 22.,  -9., -27., ..., -56., -56., -89.])}],
    'layout': {'template': '...', 'yaxis': {'ticksuffix': ' bps', 'title': {'text': '10yr - 2yr Tsry Yield'}}}
})

In [7]:
treasury_df = calculate_pct_changes(treasury_df, "10_v_2")

# Print the updated DataFrame
treasury_df

,30Y,10Y,5Y,3Y,2Y,1Y,6M,3M,10_v_2,10_v_2_3m_change,10_v_2_6m_change,10_v_2_12m_change,10_v_2_24m_change,10_v_2_3m_change_z_score,10_v_2_3m_z_score,10_v_2_6m_change_z_score,10_v_2_12m_change_z_score,10_v_2_24m_change_z_score
DATE,,,,,,,,,,,,,,,,,,
2000-01-31,6.63,6.66,6.58,6.49,6.44,6.12,5.76,5.50,0.22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-02-29,6.23,6.52,6.68,6.65,6.61,6.22,6.00,5.73,-0.09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-03-31,6.05,6.26,6.50,6.53,6.53,6.22,6.11,5.86,-0.27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-04-30,5.85,5.99,6.26,6.36,6.40,6.15,6.07,5.82,-0.41,-2.863636,NaN,NaN,NaN,-1.595332,-1.595332,NaN,NaN,NaN
2000-05-31,6.15,6.44,6.69,6.77,6.81,6.33,6.39,5.99,-0.37,3.111111,NaN,NaN,NaN,1.815625,1.815625,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-02-28,3.80,3.75,3.94,4.23,4.53,4.93,4.97,4.79,-0.78,0.278689,1.228571,-2.591837,-1.684211,0.198607,0.198607,0.324627,-0.095298,0.046776
2023-03-31,3.77,3.66,3.82,4.09,4.30,4.68,4.99,4.86,-0.64,-0.044776,0.882353,-3.909091,-1.438356,0.013943,0.013943,0.249931,-0.191919,0.057662
2023-04-30,3.68,3.46,3.54,3.76,4.02,4.68,4.99,5.07,-0.56,-0.176471,0.400000,-3.666667,-1.378378,-0.061241,-0.061241,0.145864,-0.174137,0.060318


In [8]:
# Plot z-scores
fig = go.Figure()

for column in [f"10_v_2{x}" for x in rolling_columns_change_array]:
    fig.add_trace(go.Scatter(
        x=treasury_df.index,
        y=treasury_df[column + '_z_score'],
        name=column + ' Z-Score'
    ))

fig.update_layout(
    title="Z-Scores for Percentage changes",
    xaxis=dict(title="Date", range=['2005-01-01', '2023-07-01']),
    yaxis=dict(title="Z-Score", range=[-2, 2])
)

fig.show()

In [9]:
treasury_df.iloc[[-1]]

,30Y,10Y,5Y,3Y,2Y,1Y,6M,3M,10_v_2,10_v_2_3m_change,10_v_2_6m_change,10_v_2_12m_change,10_v_2_24m_change,10_v_2_3m_change_z_score,10_v_2_3m_z_score,10_v_2_6m_change_z_score,10_v_2_12m_change_z_score,10_v_2_24m_change_z_score
DATE,,,,,,,,,,,,,,,,,,
2023-06-30,3.87,3.75,3.95,4.27,4.64,5.24,5.42,5.42,-0.89,0.390625,0.328358,-7.357143,-1.674242,0.262511,0.262511,0.130407,-0.444833,0.047218


In [10]:
z_scores_df = calculate_rolling_z_scores(treasury_df, '10_v_2')
z_scores_df.iloc[[-1]]

,10_v_2_3m_z_score,10_v_2_3m_mean,10_v_2_6m_z_score,10_v_2_6m_mean,10_v_2_12m_z_score,10_v_2_12m_mean,10_v_2_24m_z_score,10_v_2_24m_mean
DATE,,,,,,,,
2023-06-30,-1.154701,-0.67,-1.578325,-0.685,-1.609012,-0.551667,-1.348844,0.075833


In [12]:
# Plot z-scores
fig = go.Figure()

fig.add_trace(go.Scatter(x=treasury_df.index,
                                y=treasury_df["10_v_2"] * 100,
                                mode ='lines',
                                yaxis='y2'
                                )
)

for column in [f"10_v_2{x}" for x in months_array]:
    fig.add_trace(go.Scatter(
        x=z_scores_df.index,
        y=z_scores_df[column + '_z_score'],
        name=column + ' Z-Score'
    ))

    fig.add_trace(go.Scatter(
        x=z_scores_df.index,
        y=z_scores_df[column + '_mean'],
        name=column + ' Mean'
    ))

fig.update_layout(
    title="Z-Scores for Data points",
    xaxis=dict(title="Date", range=['2005-01-01', '2023-07-01']),
    yaxis=dict(title="Z-Score", range=[-3, 3]),
    yaxis2=dict(title='10s2s', overlaying='y', side='right')
)

fig.show()